# Day 20 - Race Condition

In [1]:
TEST = False

INPUT_FN = "input_test.txt"
#TEST_SOLUTION = 6

if not TEST:
    INPUT_FN = "input.txt"

In [2]:
# with open(INPUT_FN) as f:
#     s = f.read()

# patterns_block, designs_block = s.split('\n\n')

# Read and split lines
with open(INPUT_FN) as f:
    lines = [l.rstrip() for l in f]

if lines[-1]=='':
    lines=lines[:,-1]

In [3]:
lines

['#############################################################################################################################################',
 '#...#.....#...#.........#...#...#.....#...###...#.....###...#...#...#.....#...#.....#.....#.......#...#...###.............#...#...#...###...#',
 '#.#.#.###.#.#.#.#######.#.#.#.#.#.###.#.#.###.#.#.###.###.#.#.#.#.#.#.###.#.#.#.###.#.###.#.#####.#.#.#.#.###.###########.#.#.#.#.#.#.###.#.#',
 '#.#.#...#...#.#.......#.#.#.#.#.#...#...#.#...#...#...#...#.#.#.#.#...#...#.#.#...#.#.#...#.#.....#.#.#.#...#.........#...#.#.#.#.#.#.....#.#',
 '#.#.###.#####.#######.#.#.#.#.#.###.#####.#.#######.###.###.#.#.#.#####.###.#.###.#.#.#.###.#.#####.#.#.###.#########.#.###.#.#.#.#.#######.#',
 '#.#.###.#.....#.......#.#.#.#.#.#...#.....#.......#...#...#.#.#.#.#.....#...#.....#.#.#...#.#.#...#.#.#.#...#.........#.#...#.#.#.#.#.......#',
 '#.#.###.#.#####.#######.#.#.#.#.#.###.###########.###.###.#.#.#.#.#.#####.#########.#.###.#.#.#.#.#.#.#.#.###.#########.#.

Get shortest paths using `nx.single_source_shortest_path_length`

Generate the graph from data

In [4]:
nrows = len(lines)
ncols = len(lines[0])
print(f"{nrows=}, {ncols=}")

nrows=141, ncols=141


In [5]:
def get_map(iy,ix):
    l0 = lines[iy]
    return l0[ix]

Parse map to get

* list of wall tiles that may have shortcut
* edges
* start and end coordinates

In [6]:
start= None
end=None

wall_shortcuts = []
edges=[]

acceptable_node_chars = ['S','.', 'E'] 
for irow in range(nrows):
    for icol in range(ncols):
        c = get_map(irow,icol)

        # edges
        if c in acceptable_node_chars:
            #is node
            # check down or right for another node
            if irow < nrows-1:
                if get_map(irow+1,icol) in acceptable_node_chars:
                    edges.append( ((irow,icol), (irow+1, icol)) )
            if icol< ncols-1:
                if get_map(irow, icol+1) in acceptable_node_chars:
                    edges.append( ((irow,icol), (irow,icol+1)) )
            
            if c=='E':
                end = (irow,icol)
            elif c=='S':
                start = (irow,icol)
        else:
            #probably a wall
            # check around for possible shortcuts
            # horizontally
            if icol>0 and icol<ncols-1:
                if get_map(irow, icol-1) in acceptable_node_chars and \
                    get_map(irow, icol+1) in acceptable_node_chars:
                    wall_shortcuts.append( ( (irow,icol-1), (irow,icol+1)) ) 
            if irow>0 and irow<nrows-1:
                if get_map(irow-1, icol) in acceptable_node_chars and \
                    get_map(irow+1, icol) in acceptable_node_chars:
                    wall_shortcuts.append( ( (irow-1,icol), (irow+1,icol)) )


In [7]:
print(f"{start=}, {end=}")
print("edges\n", edges)
print("\nwall shortcuts\n", wall_shortcuts)


start=(105, 31), end=(93, 15)
edges
 [((1, 1), (2, 1)), ((1, 1), (1, 2)), ((1, 2), (1, 3)), ((1, 3), (2, 3)), ((1, 5), (2, 5)), ((1, 5), (1, 6)), ((1, 6), (1, 7)), ((1, 7), (1, 8)), ((1, 8), (1, 9)), ((1, 9), (2, 9)), ((1, 11), (2, 11)), ((1, 11), (1, 12)), ((1, 12), (1, 13)), ((1, 13), (2, 13)), ((1, 15), (2, 15)), ((1, 15), (1, 16)), ((1, 16), (1, 17)), ((1, 17), (1, 18)), ((1, 18), (1, 19)), ((1, 19), (1, 20)), ((1, 20), (1, 21)), ((1, 21), (1, 22)), ((1, 22), (1, 23)), ((1, 23), (2, 23)), ((1, 25), (2, 25)), ((1, 25), (1, 26)), ((1, 26), (1, 27)), ((1, 27), (2, 27)), ((1, 29), (2, 29)), ((1, 29), (1, 30)), ((1, 30), (1, 31)), ((1, 31), (2, 31)), ((1, 33), (2, 33)), ((1, 33), (1, 34)), ((1, 34), (1, 35)), ((1, 35), (1, 36)), ((1, 36), (1, 37)), ((1, 37), (2, 37)), ((1, 39), (2, 39)), ((1, 39), (1, 40)), ((1, 40), (1, 41)), ((1, 41), (2, 41)), ((1, 45), (2, 45)), ((1, 45), (1, 46)), ((1, 46), (1, 47)), ((1, 47), (2, 47)), ((1, 49), (2, 49)), ((1, 49), (1, 50)), ((1, 50), (1, 51)), ((

Get default route

In [8]:
import networkx as nx

G = nx.Graph(edges)

In [9]:
lengths = nx.single_source_shortest_path_length(G,start)
lengths

{(105, 31): 0,
 (105, 30): 1,
 (105, 29): 2,
 (105, 28): 3,
 (105, 27): 4,
 (105, 26): 5,
 (105, 25): 6,
 (105, 24): 7,
 (105, 23): 8,
 (105, 22): 9,
 (105, 21): 10,
 (105, 20): 11,
 (105, 19): 12,
 (105, 18): 13,
 (105, 17): 14,
 (105, 16): 15,
 (105, 15): 16,
 (105, 14): 17,
 (105, 13): 18,
 (106, 13): 19,
 (107, 13): 20,
 (107, 12): 21,
 (107, 11): 22,
 (108, 11): 23,
 (109, 11): 24,
 (109, 10): 25,
 (109, 9): 26,
 (108, 9): 27,
 (107, 9): 28,
 (106, 9): 29,
 (105, 9): 30,
 (104, 9): 31,
 (103, 9): 32,
 (103, 10): 33,
 (103, 11): 34,
 (102, 11): 35,
 (101, 11): 36,
 (101, 10): 37,
 (101, 9): 38,
 (100, 9): 39,
 (99, 9): 40,
 (99, 8): 41,
 (99, 7): 42,
 (98, 7): 43,
 (97, 7): 44,
 (97, 6): 45,
 (97, 5): 46,
 (98, 5): 47,
 (99, 5): 48,
 (99, 4): 49,
 (99, 3): 50,
 (99, 2): 51,
 (99, 1): 52,
 (100, 1): 53,
 (101, 1): 54,
 (101, 2): 55,
 (101, 3): 56,
 (101, 4): 57,
 (101, 5): 58,
 (101, 6): 59,
 (101, 7): 60,
 (102, 7): 61,
 (103, 7): 62,
 (103, 6): 63,
 (103, 5): 64,
 (103, 4): 65,
 (

In [10]:
from collections import defaultdict

In [11]:
# help(defaultdict)

In [12]:
ps_save_counter = defaultdict(int)

In [13]:
for wsc0 in wall_shortcuts:
    node0, node1 = wsc0

    # get the lenghts of each
    pico0 = lengths[node0]
    pico1 = lengths[node1]

    absdiff = abs(pico0-pico1)
    if absdiff>2:
        save = absdiff-2
        # can save picoseconds if going through this wall
        print (f"can save {save}, going through {node0} to/from {node1}")

        ps_save_counter[save]+=1


can save 24, going through (1, 3) to/from (1, 5)
can save 4, going through (1, 9) to/from (1, 11)
can save 160, going through (1, 13) to/from (1, 15)
can save 12, going through (1, 23) to/from (1, 25)
can save 12, going through (1, 27) to/from (1, 29)
can save 48, going through (1, 31) to/from (1, 33)
can save 4, going through (1, 37) to/from (1, 39)
can save 4, going through (1, 47) to/from (1, 49)
can save 32, going through (1, 59) to/from (1, 61)
can save 16, going through (1, 63) to/from (1, 65)
can save 4, going through (1, 67) to/from (1, 69)
can save 552, going through (1, 73) to/from (1, 75)
can save 12, going through (1, 77) to/from (1, 79)
can save 52, going through (1, 83) to/from (1, 85)
can save 24, going through (1, 89) to/from (1, 91)
can save 412, going through (1, 97) to/from (1, 99)
can save 48, going through (1, 101) to/from (1, 103)
can save 32, going through (1, 121) to/from (1, 123)
can save 24, going through (1, 125) to/from (1, 127)
can save 16, going through (1

In [14]:
ps_save_counter

defaultdict(int,
            {24: 128,
             4: 959,
             160: 23,
             12: 318,
             48: 59,
             32: 98,
             16: 220,
             552: 2,
             52: 48,
             412: 3,
             2: 959,
             22: 66,
             158: 11,
             6: 265,
             8: 426,
             10: 195,
             46: 29,
             30: 42,
             14: 137,
             550: 2,
             50: 25,
             410: 3,
             20: 172,
             156: 21,
             44: 71,
             584: 1,
             28: 112,
             548: 4,
             408: 3,
             164: 15,
             18: 99,
             582: 1,
             26: 62,
             544: 4,
             42: 34,
             140: 18,
             36: 90,
             580: 1,
             540: 2,
             392: 4,
             40: 80,
             144: 13,
             34: 41,
             546: 3,
             520: 3,
             522: 2,
    

In [15]:
at_least_ps = 100

In [16]:
res_count = 0
for k,v in ps_save_counter.items():
    if k>=at_least_ps:
        res_count+=v

print (res_count)

1463


OK, 1463 is the correct answer

# Part 2

Wall cheats that can last up to 20ps!!

Probably best to treat wall tiles as a graph network of its own, that connect edge tiles.

Reparse map, but this time capture wall edges. Don't include nodes on the track

In [17]:
wall_edges = set()

for irow in range(0,nrows):
    for icol in range(0,ncols):
        c = get_map(irow,icol)

        if irow<nrows-1:
            c1 = get_map(irow+1,icol)

            if c == '#' and c1=='#':
                wall_edges.add(((irow,icol),(irow+1,icol)))
        
        if icol<ncols-1:
            c1 = get_map(irow,icol+1)

            if c == '#' and c1=='#':
                wall_edges.add(((irow,icol),(irow,icol+1)))

In [18]:
print("wall_edges\n", wall_edges)
#print("wall_node_margins\n", wall_node_margins)

wall_edges
 {((91, 38), (92, 38)), ((37, 72), (38, 72)), ((101, 58), (102, 58)), ((136, 42), (137, 42)), ((101, 33), (101, 34)), ((52, 124), (53, 124)), ((9, 20), (10, 20)), ((41, 0), (42, 0)), ((66, 113), (67, 113)), ((135, 107), (135, 108)), ((49, 64), (50, 64)), ((116, 40), (117, 40)), ((101, 102), (102, 102)), ((28, 104), (29, 104)), ((30, 34), (31, 34)), ((50, 128), (51, 128)), ((51, 98), (52, 98)), ((0, 102), (0, 103)), ((2, 40), (3, 40)), ((97, 27), (97, 28)), ((0, 14), (0, 15)), ((14, 100), (15, 100)), ((94, 13), (94, 14)), ((134, 31), (134, 32)), ((57, 0), (58, 0)), ((25, 46), (26, 46)), ((2, 84), (3, 84)), ((120, 36), (121, 36)), ((56, 112), (57, 112)), ((30, 8), (30, 9)), ((29, 106), (30, 106)), ((53, 6), (54, 6)), ((13, 6), (14, 6)), ((32, 114), (33, 114)), ((76, 26), (77, 26)), ((84, 15), (84, 16)), ((73, 12), (74, 12)), ((102, 19), (103, 19)), ((61, 138), (62, 138)), ((112, 90), (113, 90)), ((38, 27), (38, 28)), ((91, 130), (92, 130)), ((54, 120), (54, 121)), ((132, 26), 

Now, iterate through the track nodes.

Need to create a list of neighbouring wall nodes.

Maybe create a copy of the wall graph and just adds the new nodes. Then it calculates the shortest path length

Check if there is a wall shortcut between them. If there is and is less or equal than 20 consider it. Check against the normal route. if the shortcut gives less, then is a valid shortcut.

In [19]:
GWall = nx.Graph(wall_edges) # default

In [20]:
# from itertools import combinations

# ps_save_counter2 = defaultdict(int)

# for node0, node1 in combinations(G.nodes,2):
#     GWall2 = GWall.copy()
#     # Generate new connectivity with walls around
    
#     for n in [node0, node1]:
#         for dr,dc in [(1,0),(-1,0),(0,1),(0,-1)]:
#             check_wall_at = (n[0]+dr, n[1]+dc)
#             if check_wall_at in GWall.nodes:
#                 GWall2.add_edge( n, check_wall_at )

#     if node0 not in GWall2.nodes or node1 not in GWall2.nodes:
#         #cannot build a shortcut
#         continue

#     pico0 = lengths[node0]
#     pico1 = lengths[node1]

#     absdiff = abs(pico0-pico1)
#     if absdiff<=2:
#         continue # don't bother

#     #compute this diffference using shortest_path_length and compare
#     # track_len = nx.shortest_path_length(G, node0,node1)
#     # if track_len != absdiff:
#     #     print(f"MISMATCH {track_len=}, {absdiff=}")

#     if nx.has_path(GWall2, node0, node1):
#         # get the lenghts of each
#         print (f"Has wall path between {node0} and {node1}")

#         wall_length = nx.shortest_path_length(GWall2, node0,node1)

#         print(f"{wall_length=}, {absdiff=}")
#         if wall_length<=20:
#             #Valid shortcut
#             save = absdiff-wall_length

#             if save>0:
#                 # it is a shortcut
#                 print (f"Can save {save}, going through {node0} to/from {node1}")

#                 ps_save_counter2[save]+= 1


In [22]:
def printd(s):
    if TEST:
        print(s)

In [23]:
# Try to use manhatan distance between nodes
from itertools import combinations

ps_save_counter2 = defaultdict(int)

for node0, node1 in combinations(G.nodes,2):
    # Generate new connectivity with walls around
    
    manh_dist = abs( node0[0]-node1[0]) + abs(node0[1]-node1[1])

    if manh_dist>20:
        continue

    pico0 = lengths[node0]
    pico1 = lengths[node1]

    absdiff = abs(pico0-pico1)

    if absdiff<=2:
        continue # don't bother

    save = absdiff-manh_dist

    if save>0:
        # it is a shortcut
        printd (f"Can save {save}, going through {node0} to/from {node1}")

        ps_save_counter2[save]+= 1


In [24]:
ps_save_counter2

defaultdict(int,
            {24: 20954,
             22: 10968,
             28: 18914,
             26: 10318,
             188: 6637,
             186: 3775,
             20: 21611,
             184: 6164,
             18: 11537,
             16: 24582,
             14: 12460,
             182: 3848,
             180: 6459,
             178: 3879,
             176: 6096,
             12: 28403,
             10: 13491,
             36: 16711,
             34: 9319,
             32: 17373,
             30: 9738,
             168: 6904,
             166: 4025,
             8: 30657,
             164: 6894,
             6: 14731,
             56: 13120,
             54: 7471,
             4: 42051,
             52: 13733,
             2: 16655,
             40: 15798,
             38: 8962,
             50: 7832,
             48: 14747,
             136: 8082,
             134: 4603,
             132: 8187,
             130: 4750,
             128: 7889,
             126: 4812,
        

In [25]:
for t in range (50,77,2):
    print(f"cheats {ps_save_counter2[t]} picoseconds {t}")

cheats 7832 picoseconds 50
cheats 13733 picoseconds 52
cheats 7471 picoseconds 54
cheats 13120 picoseconds 56
cheats 7189 picoseconds 58
cheats 12797 picoseconds 60
cheats 6989 picoseconds 62
cheats 11661 picoseconds 64
cheats 6875 picoseconds 66
cheats 12200 picoseconds 68
cheats 6659 picoseconds 70
cheats 11531 picoseconds 72
cheats 6419 picoseconds 74
cheats 11684 picoseconds 76


This appears to be the correct answer, however there is no garantee the number of shortcuts in a single cheats is single.

In [26]:
at_least_ps2 = 100
if TEST:
    at_least_ps2 = 50

at_least_ps2

100

In [27]:
res_count2 = 0
for k,v in ps_save_counter2.items():
    if k>=at_least_ps2:
        res_count2+=v

print (res_count2)

985332
